# Image classifiers // VGG16
1. Setup
2. Data transformations
3. Data load
### 1. Setup // Import modules

In [ ]:
import copy
import os
import shutil
import time

import pandas as pd

from image_utils import copy_image_to_class_folders

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms

### 2. Image data preparation
Organize processed images into class-specific directories for model training.

This cell takes preprocessed images and copies them into a structured directory hierarchy suitable for training image classification models. It performs a train/validation split (80/20 by default) and organizes images by their product type code into separate class folders.

In [ ]:
#df_image_train = pd.read_csv("df_image_train.csv", index_col="productid")
#copy_image_to_class_folders(df_image_train) # Duration 26apr2025

### 3. Data transformations

In [ ]:
# 2. Define data transformations
data_transforms = {
    'image_train_vgg16': transforms.Compose([
#        transforms.Resize((224, 224)),  # VGG16 requires 224x224 input
        transforms.RandomHorizontalFlip(),  # Data augmentation
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet stats
    ]),
    'image_val_vgg16': transforms.Compose([
#        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

### 3. Load data

In [ ]:
# 3. Load data
data_dir = 'images/image_train_vgg16'  
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['image_train_vgg16', 'image_val_vgg16']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['image_train_vgg16', 'image_val_vgg16']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['image_train_vgg16', 'image_val_vgg16']}

class_names = image_datasets['image_train_vgg16'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### 4. Load model

In [ ]:
# Load pre-trained VGG16 model
model_ft = models.vgg16(weights='IMAGENET1K_V1')

### 5. Modify the classifier

In [ ]:
# Freeze all parameters in the features (convolutional) layers
for param in model_ft.features.parameters():
    param.requires_grad = False

# Replace the final layer in the classifier
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs, 27)  # Replace with 27 outputs for your classes

model_ft = model_ft.to(device)  # Move model to GPU if available

### 6. Loss Function and Optimizer

In [ ]:
# Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()

# Only optimize parameters that require gradients
optimizer_ft = optim.SGD(filter(lambda p: p.requires_grad, model_ft.parameters()), 
                         lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

### 7. Train and evaluate the model

In [ ]:
# Training function
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    # Update these keys to match your actual dataloader dictionary keys
    train_key = 'image_train_vgg16'  # Replace with your training data key
    val_key = 'image_val_vgg16'     # Replace with your validation/test data key
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in [train_key, val_key]:
            if phase == train_key:
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
                
            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # Zero the parameter gradients
                optimizer.zero_grad()
                
                # Forward pass
                # track history if only in train
                with torch.set_grad_enabled(phase == train_key):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # Backward + optimize only if in training phase
                    if phase == train_key:
                        loss.backward()
                        optimizer.step()
                
                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == train_key:
                scheduler.step()
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            # Deep copy the model if best
            if phase == val_key and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        print()
    
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')
    
    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# Train and evaluate the model
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=1)

### 8. Save the trained model

In [ ]:
torch.save(model_ft.state_dict(), 'vgg16_transfer_model.pth')